In [1]:
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 
import glob

plt.rcParams['figure.figsize'] = [9, 6]

### LoadFrames 

In [2]:
Frames = []
Path = glob.glob("./inputs/P3/frames/*.png")
Path.sort()
for img in Path:
    img = cv2.imread(img)
    Frames.append(img)
Frames = np.array(Frames)
MeanBack = []
MedianBack = []

### Load Tests

In [3]:
Tests = []
Path = glob.glob("./inputs/P3/tests/*.png")
Path.sort()
for img in Path:
    img = cv2.imread(img)
    Tests.append(img)

### Estimate background

In [4]:
def estimate_background(n=20,med=False):
    back = np.zeros_like(Frames[0])
    for row in range(Frames[0].shape[0]):
        for col in range(Frames[0].shape[1]):
            items = Frames[:n,row,col,:]
            if not med :
                back[row,col,:] = np.mean(np.array(items),axis=0)
            else :
                back[row,col,:] = np.median(np.array(items),axis=0)
                
    return back

### A

In [5]:
for n in [2,5,10,20]:
    print("Proc: ",n)
    MeanBack.append(estimate_background(n,False))
    print("Current len: ",len(MeanBack))

Proc:  2
Current len:  1
Proc:  5
Current len:  2
Proc:  10
Current len:  3
Proc:  20
Current len:  4


In [6]:
tmp = cv2.vconcat(MeanBack)
cv2.imwrite("./outputs/P03/a.png",tmp)

True

### B

In [11]:
for i in range(2): 
    test = Tests[i] 
    results = []
    for back in MeanBack :
        results.append(cv2.subtract(back,test))
    tmp = cv2.vconcat(results)
    tmp = cv2.cvtColor(tmp,cv2.COLOR_BGR2GRAY)
    cv2.imwrite("./outputs/P03/b-test-{0}.png".format(i+1),tmp)
        

### C

In [96]:
for i in range(2): 
    test = Tests[i] 
    results = []
    for back in MeanBack :
        _sub = cv2.subtract(back,test)
        #_sub = cv2.cvtColor(_sub,cv2.COLOR_BGR2GRAY)
        thresh = np.histogram(_sub.flatten())[1][2]
        _, mask = cv2.threshold(_sub, thresh, 255, cv2.THRESH_BINARY)
        results.append(mask)
        print(thresh)
        
    cv2.imwrite("./outputs/P03/c-test-{0}.png".format(i+1),cv2.vconcat(results))


36.0
32.6
32.0
33.8
41.4
41.4
37.4
32.8


### D

In [97]:
for i in range(2): 
    test = Tests[i] 
    results = []
    for back in MeanBack :
        _sub = cv2.subtract(back,test)
        _sub = cv2.cvtColor(_sub,cv2.COLOR_BGR2GRAY)
        thresh = np.histogram(_sub.flatten())[1][2]
        _, mask = cv2.threshold(_sub, thresh, 255, cv2.THRESH_BINARY)
        masked = cv2.bitwise_and(test, test, mask=mask)
        results.append(masked)
        
    cv2.imwrite("./outputs/P03/d-test-{0}.png".format(i+1),cv2.vconcat(results))




### E

In [39]:
for n in [2,5,10,20]:
    print("Proc: ",n)
    MedianBack.append(estimate_background(n,True))
    print("Current len: ",len(MedianBack))
    
tmp = cv2.vconcat(MedianBack)
cv2.imwrite("./outputs/P03/E-a.png",tmp)

for i in range(2): 
    test = Tests[i] 
    results = []
    for back in MedianBack :
        results.append(cv2.subtract(back,test))
    tmp = cv2.vconcat(results)
    tmp = cv2.cvtColor(tmp,cv2.COLOR_BGR2GRAY)
    cv2.imwrite("./outputs/P03/E-b-test-{0}.png".format(i+1),tmp)
        
    
for i in range(2): 
    test = Tests[i] 
    results = []
    for back in MedianBack :
        _sub = cv2.subtract(back,test)
        _sub = cv2.cvtColor(_sub,cv2.COLOR_BGR2GRAY)
        thresh = np.histogram(_sub.flatten())[1][2]
        _, mask = cv2.threshold(_sub, thresh, 255, cv2.THRESH_BINARY)
        results.append(mask)
        
    cv2.imwrite("./outputs/P03/E-c-test-{0}.png".format(i+1),cv2.vconcat(results))
    
for i in range(2): 
    test = Tests[i] 
    results = []
    for back in MedianBack :
        _sub = cv2.subtract(back,test)
        _sub = cv2.cvtColor(_sub,cv2.COLOR_BGR2GRAY)
        thresh = np.histogram(_sub.flatten())[1][2]
        _, mask = cv2.threshold(_sub, thresh, 255, cv2.THRESH_BINARY)
        masked = cv2.bitwise_and(test, test, mask=mask)
        results.append(masked)
        
    cv2.imwrite("./outputs/P03/E-d-test-{0}.png".format(i+1),cv2.vconcat(results))





Proc:  2
Current len:  1
Proc:  5
Current len:  2
Proc:  10
Current len:  3
Proc:  20
Current len:  4


### F

In [62]:

def counting_pedestrians_avg():
    for i in range(2): 
        print("Proc test #{0}:".format(i+1))
        test = Tests[i] 
        show_result = []
        for back in MedianBack :
            _sub = cv2.subtract(back,test)
            _sub = cv2.cvtColor(_sub,cv2.COLOR_BGR2GRAY)
            thresh = np.histogram(_sub.flatten())[1][2]
            _, mask = cv2.threshold(_sub, thresh, 255, cv2.THRESH_BINARY)
            mask =cv2.blur(mask,(9,9))
            _, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
            show_result.append(mask)
            
            ret, labels = cv2.connectedComponents(mask, connectivity=4)
            unique, counts = np.unique(labels, return_counts=True)

            print("-> Count of pedestrians extracted(Averaging filter): ",sum(counts>300))
            
        cv2.imwrite("./outputs/P03/f-test-averaging-{0}.png".format(i+1),cv2.vconcat(show_result))
        
        
counting_pedestrians_avg()    

Proc test #1:
-> Count of pedestrians extracted(Averaging filter):  18
-> Count of pedestrians extracted(Averaging filter):  16
-> Count of pedestrians extracted(Averaging filter):  16
-> Count of pedestrians extracted(Averaging filter):  15
Proc test #2:
-> Count of pedestrians extracted(Averaging filter):  18
-> Count of pedestrians extracted(Averaging filter):  18
-> Count of pedestrians extracted(Averaging filter):  18
-> Count of pedestrians extracted(Averaging filter):  20


In [64]:
def counting_pedestrians_gaussian():
    for i in range(2): 
        print("Proc test #{0}:".format(i+1))
        test = Tests[i] 
        show_result = []
        for back in MedianBack :
            _sub = cv2.subtract(back,test)
            _sub = cv2.cvtColor(_sub,cv2.COLOR_BGR2GRAY)
            thresh = np.histogram(_sub.flatten())[1][2]
            _, mask = cv2.threshold(_sub, thresh, 255, cv2.THRESH_BINARY)
            
            mask =cv2.GaussianBlur(mask,(9,9),0)
            _, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
            show_result.append(mask)
            
            ret, labels = cv2.connectedComponents(mask, connectivity=4)
            unique, counts = np.unique(labels, return_counts=True)

            print("-> Count of pedestrians extracted(Gaussian filter): ",sum(counts>300))
            
        cv2.imwrite("./outputs/P03/f-test-Gaussian-{0}.png".format(i+1),cv2.vconcat(show_result))
        
        
counting_pedestrians_gaussian()    

Proc test #1:
-> Count of pedestrians extracted(Gaussian filter):  20
-> Count of pedestrians extracted(Gaussian filter):  16
-> Count of pedestrians extracted(Gaussian filter):  16
-> Count of pedestrians extracted(Gaussian filter):  16
Proc test #2:
-> Count of pedestrians extracted(Gaussian filter):  21
-> Count of pedestrians extracted(Gaussian filter):  20
-> Count of pedestrians extracted(Gaussian filter):  19
-> Count of pedestrians extracted(Gaussian filter):  21


In [65]:
def counting_pedestrians_Median():
    for i in range(2): 
        print("Proc test #{0}:".format(i+1))
        test = Tests[i] 
        show_result = []
        for back in MedianBack :
            _sub = cv2.subtract(back,test)
            _sub = cv2.cvtColor(_sub,cv2.COLOR_BGR2GRAY)
            thresh = np.histogram(_sub.flatten())[1][2]
            _, mask = cv2.threshold(_sub, thresh, 255, cv2.THRESH_BINARY)
            
            mask =cv2.medianBlur(mask,5)
            _, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
            show_result.append(mask)
            
            ret, labels = cv2.connectedComponents(mask, connectivity=4)
            unique, counts = np.unique(labels, return_counts=True)

            print("-> Count of pedestrians extracted(Median filter): ",sum(counts>300))
            
        cv2.imwrite("./outputs/P03/f-test-Median-{0}.png".format(i+1),cv2.vconcat(show_result))
        
        
counting_pedestrians_Median()    

Proc test #1:
-> Count of pedestrians extracted(Median filter):  22
-> Count of pedestrians extracted(Median filter):  16
-> Count of pedestrians extracted(Median filter):  18
-> Count of pedestrians extracted(Median filter):  17
Proc test #2:
-> Count of pedestrians extracted(Median filter):  24
-> Count of pedestrians extracted(Median filter):  21
-> Count of pedestrians extracted(Median filter):  21
-> Count of pedestrians extracted(Median filter):  23
